In [2]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt
import json
from urllib.request import urlopen

with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
    Brazil_estados = json.load(response)

gdf_mun = gpd.read_file('geojson/geojson_mun.json')

state_id_map = {}
for feature in Brazil_estados ['features']:
    feature['id'] = feature['properties']['name']
    state_id_map[feature['properties']['sigla']] = feature['id']

df_municipios = pd.read_csv('../../Dados Tratados/Atlas_Municipios_SP.csv')

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

gdf = gdf_mun.copy()

gdf["name"] = gdf["name"].str.upper()
df_municipios["Local"] = df_municipios["Local"].str.upper()

anos = df_municipios["Ano"].unique()

# Normalizar os valores de "Morte5" para a barra de cores
df_municipios["Morte5"] = df_municipios["Morte5"].str.replace(",", ".").astype(float)
vmin = df_municipios["Morte5"].min()
vmax = df_municipios["Morte5"].max()
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = plt.cm.OrRd

for ano in anos:
    df_ano = df_municipios[df_municipios["Ano"] == ano]
    
    # Mesclar dados com o GeoDataFrame
    gdf_ano = gdf.merge(df_ano, left_on="name", right_on="Local", how="left")
    
    # Criar figura e eixo
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    # Plotar o mapa
    gdf_ano.plot(column="Morte5", cmap=cmap, linewidth=0.8, edgecolor="black",
                 ax=ax, missing_kwds={"color": "lightgrey"}, norm=norm)

    # Adicionar barra de cores ao lado
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Necessário para evitar erro
    cbar = fig.colorbar(sm, ax=ax, fraction=0.02, pad=0.02)
    cbar.set_label("Mortalidade (Morte5)")

    # Ajustes visuais
    ax.set_title(f"Mortalidade - menores de 5 anos {ano}")
    ax.axis("off")

    # Salvar cada ano como uma imagem separada
    plt.savefig(f"mapa_mortalidade_{ano}.png", dpi=300, bbox_inches="tight")
    plt.close()

print("Mapas gerados e salvos com sucesso!")

Mapas gerados e salvos com sucesso!


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import branca

# Carregar os dados geoespaciais
gdf_mun = gpd.read_file('geojson/geojson_mun.json')

# Carregar os dados dos municípios
df_municipios = pd.read_csv('../../Dados Tratados/Atlas_Municipios_SP.csv')

gdf_mun["name"] = gdf_mun["name"].str.upper()
df_municipios["Local"] = df_municipios["Local"].str.upper()

# Converter "Morte5" para float
df_municipios["Morte5"] = df_municipios["Morte5"].str.replace(",", ".").astype(float)
anos = df_municipios["Ano"].unique()

# Criar os mapas interativos para cada ano
for ano in anos:
    df_ano = df_municipios[df_municipios["Ano"] == ano]
    gdf_ano = gdf_mun.merge(df_ano, left_on="name", right_on="Local", how="left")
    
    # Filtrar apenas as linhas com geometrias válidas
    gdf_ano = gdf_ano[gdf_ano.geometry.notnull()]
    
    # Criar um mapa centralizado em São Paulo
    centro_sp = [-23.55052, -46.633308]
    mapa = folium.Map(location=centro_sp, zoom_start=6)
    
    # Criar escala de cores
    vmin, vmax = df_ano["Morte5"].min(), df_ano["Morte5"].max()
    colormap = branca.colormap.LinearColormap(['yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)
    colormap.caption = "Mortalidade Infantil (Morte5)"
    
    # Adicionar polígonos ao mapa
    for _, row in gdf_ano.iterrows():
        cor = colormap(row["Morte5"]) if not pd.isnull(row["Morte5"]) else "lightgrey"
        
        folium.GeoJson(
            row.geometry,
            style_function=lambda x, cor=cor: {"fillColor": cor, "color": "black", "weight": 0.5, "fillOpacity": 0.7},
            tooltip=folium.Tooltip(f"{row['name']}: {row['Morte5']:.2f}" if not pd.isnull(row['Morte5']) else row['name'])
        ).add_to(mapa)
    
    colormap.add_to(mapa)
    mapa

print("Mapas interativos gerados e salvos com sucesso!")


c:\Users\joaov\anaconda3\Lib\site-packages\geopandas\geoseries.py:860: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
c:\Users\joaov\anaconda3\Lib\site-packages\geopandas\geoseries.py:860: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get bac

Mapas interativos gerados e salvos com sucesso!
